In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

train_set_x = pd.read_csv('../data/train_set_x_reduced.csv')
train_set_y = pd.read_csv('../data/train_set_y.csv')
test_set_x = pd.read_csv('../data/test_set_x_reduced.csv')
test_set_y = pd.read_csv('../data/test_set_y.csv')
pca_components = pd.read_csv('../data/pca_components.csv')


FileNotFoundError: [Errno 2] File ../data/pca_components.csv does not exist: '../data/pca_components.csv'

In [ ]:
# Random search cross validation

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestClassifier

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=100, stop=2000, num=20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = range(1, 21)
# Minimum number of samples required to split a node
min_samples_split = range(1, 11)
# Minimum number of samples required at each leaf node
min_samples_leaf = range(1, 6)
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


RFC = RandomForestClassifier()
# Use the random grid to search for best hyperparameters
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
RFC_random_CV = RandomizedSearchCV(estimator=RFC, param_distributions=random_grid,
                               n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)
# Fit the random search model
search = RFC_random_CV.fit(train_set_x, train_set_y)
print('Hyperparameter tuning:', search.best_params_)
